# SQL - Project practice, CIA world factbook 

### Intro
This project uses data from the [CIA World Factbook](https://www.cia.gov/the-world-factbook/) which contains demographic information such as populations for each country, land and water by area, birth and death rates, etc.


**Columns**
- `name` — the name of the country.
- `area` — the country's total area (both land and water).
- `area_land` — the country's land area in square kilometers.
- `area_water` — the country's waterarea in square kilometers.
`population` — the country's population.
- `population_growth` — the country's population growth as a percentage.
- `birth_rate` — the country's birth rate, or the number of - births per year per 1,000 people.
- `death_rate` — the country's death rate, or the number of death per year per 1,000 people.

### Aims
A basic walkthrough with SQL, selecting and performing some summary statistics on a dataset. 


In [66]:
# connect notebook to database file

In [67]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [68]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


--selecting first five rows of facts table in database

In [69]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


# Summary statistics - initial observations

I'll start with some initial calculations in order to find any outlier countries.

In [70]:
#single query to return:
#min max population, min max population growth

In [71]:
%%sql

SELECT MIN(population) AS min_population,
MAX(population) AS max_population,
MIN(population_growth) AS min_growth,
MAX(population_growth) AS max_growth
  FROM facts
 LIMIT 10;

Done.


min_population,max_population,min_growth,max_growth
0,7256490011,0.0,4.02


### Observations 
- countries listed with a population of 0
- country listed with a max pop. of 7.2 billion

These results are obvious outliers. Will need to filter based on these criteria in order to find out which country these results belong to.

**--writing a query to return countries with minimum population, and a query to return countries with the max population**

In [72]:
%%sql

SELECT *
  FROM facts
 WHERE population = 0;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


In [73]:
%%sql

SELECT *
  FROM facts
 WHERE population > 7000000000;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


above could also be written as:

```
SELECT *
  FROM facts
 WHERE population == (SELECT MAX(population)
                        FROM facts
                     );```
                     
Or MIN(population) to find out min value

### Observations
The entry corresponding to the maximum population value is actually a value for the whole world - which does make sense but needs to be excluded from future queries, otherwise it will be skew the results.

The minimum population value is an entry for Anarctica. 

The next steps will be to recalculate the summary statistics e.g. min/max population & growth, while excluding the 'World' entry.

**--calculate min/max population & growth, excluding 'World' entry, either <> or != operator**

In [74]:
%%sql 

SELECT MIN(population) AS min_population,
MAX(population) AS max_population,
MIN(population_growth) AS min_growth,
    MAX(population_growth) AS max_growth
FROM facts
 WHERE name <> 'World'

Done.


min_population,max_population,min_growth,max_growth
0,1367485388,0.0,4.02


# Summary statistics - density
In order to explore the density of each country, we'll have to perform some summary statistics on the two columns which impact density: population and area.

**--calculating average values for population and area**

In [75]:
%%sql

SELECT AVG(population) AS average_population, AVG(area) AS average_area
  FROM facts
 WHERE name <> 'World';

Done.


average_population,average_area
32242666.56846473,555093.546184739


### Observations
1. average population is approximately 32 million
2. average area is approximately 555 km^2

Now, i'll explore countries where population is above average, but the area is below average.

In [76]:
%%sql

SELECT *
  FROM facts
 WHERE population > (SELECT AVG(population)
                       FROM facts
                      WHERE name <> 'World') AND area < (SELECT AVG(area)
                                                 FROM facts
                                                WHERE name <> 'World');

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


# Summary statistics - death and birth rate

Looking at countries with a higher death rate than birth rate.

In [77]:
%%sql

SELECT MIN(birth_rate) as min_birth, MAX(birth_rate) AS max_birth, MIN(death_rate) AS min_death, MAX(death_rate) as max_death
  FROM facts;

Done.


min_birth,max_birth,min_death,max_death
6.65,45.45,1.53,14.89


In [78]:
%%sql

SELECT *
  FROM facts
 WHERE birth_rate > 40;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
27,uv,Burkina Faso,274200,273800,400,18931686,3.03,42.03,11.72,0.0
29,by,Burundi,27830,25680,2150,10742276,3.28,42.01,9.27,0.0
106,mi,Malawi,118484,94080,24404,17964697,3.32,41.56,8.41,0.0
109,ml,Mali,1240192,1220190,20002,16955536,2.98,44.99,12.89,2.26
128,ng,Niger,None,1266700,300,18045729,3.25,45.45,12.42,0.56
160,so,Somalia,637657,627337,10320,10616380,1.83,40.45,13.62,8.49
182,ug,Uganda,241038,197100,43938,37101745,3.24,43.79,10.69,0.74
194,za,Zambia,752618,743398,9220,15066266,2.88,42.13,12.67,0.68


In [79]:
%%sql

SELECT *
  FROM facts
 WHERE death_rate < 2;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
141,qa,Qatar,11586,11586,0,2194817,3.07,9.84,1.53,22.39
184,ae,United Arab Emirates,83600,83600,0,5779760,2.58,15.43,1.97,12.36


In [80]:
%%sql
SELECT *, ROUND(death_rate - birth_rate, 3) AS birth_death_dif
  FROM facts
 WHERE birth_rate < death_rate
ORDER BY birth_death_dif DESC

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,birth_death_dif
26,bu,Bulgaria,110879,108489,2390,7186893,0.58,8.92,14.44,0.29,5.52
153,ri,Serbia,77474,77474,0,7176794,0.46,9.08,13.66,0.0,4.58
96,lg,Latvia,64589,62249,2340,1986705,1.06,10.0,14.31,6.26,4.31
102,lh,Lithuania,65300,62680,2620,2884433,1.04,10.1,14.27,6.27,4.17
183,up,Ukraine,603550,579330,24220,44429471,0.6,10.72,14.46,2.25,3.74
75,hu,Hungary,93028,89608,3420,9897541,0.22,9.16,12.73,1.33,3.57
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24,2.95
158,si,Slovenia,20273,20151,122,1983412,0.26,8.42,11.37,0.37,2.95
142,ro,Romania,238391,229891,8500,21666350,0.3,9.14,11.9,0.24,2.76
44,hr,Croatia,56594,55974,620,4464844,0.13,9.45,12.18,1.39,2.73


### Observations
Bulgaria has the worst birth/death discrepancy (where death rate is higher than birth), followed by Serbia and Latvia. Interestingly, most of the countries that appear on the list (especially higher up) are located in Eastern Europe and are ex-soviet countries. 

Japan is the only Non-European country to appear on the list, if we exclude the small island of Saint Pierre and Miquelon (technically located in North America, but a French colony).

Many countries with very high birth rates (defined as > 40 birth_rate) are located in Africa.

# Summary statistics - growth rate

In [81]:
%%sql

SELECT *
  FROM facts
 ORDER BY population_growth DESC
 LIMIT 10;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47
106,mi,Malawi,118484,94080,24404,17964697,3.32,41.56,8.41,0.0
29,by,Burundi,27830,25680,2150,10742276,3.28,42.01,9.27,0.0
128,ng,Niger,None,1266700,300,18045729,3.25,45.45,12.42,0.56
182,ug,Uganda,241038,197100,43938,37101745,3.24,43.79,10.69,0.74
141,qa,Qatar,11586,11586,0,2194817,3.07,9.84,1.53,22.39
27,uv,Burkina Faso,274200,273800,400,18931686,3.03,42.03,11.72,0.0
109,ml,Mali,1240192,1220190,20002,16955536,2.98,44.99,12.89,2.26
219,cw,Cook Islands,236,236,0,9838,2.95,14.33,8.03,None
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62


In [82]:
%%sql

SELECT *
  FROM facts
 ORDER BY population_growth
 LIMIT 10;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
92,kv,Kosovo,10887,10887,0,1870981,None,None,None,None
198,at,Ashmore and Cartier Islands,5,5,0,None,None,None,None,None
201,cr,Coral Sea Islands,3,3,0,None,None,None,None,None
202,hm,Heard Island and McDonald Islands,412,412,0,None,None,None,None,None
208,ip,Clipperton Island,6,6,0,None,None,None,None,None
210,fs,French Southern and Antarctic Lands,None,None,None,None,None,None,None,None
212,tb,Saint Barthelemy,None,None,None,7237,None,None,None,None
213,rn,Saint Martin,54,54,None,31754,None,None,None,None
222,bv,Bouvet Island,49,49,0,None,None,None,None,None
223,jn,Jan Mayen,377,377,0,None,None,None,None,None


Many of the above results include small island colonies and are therefore not reflective of life in countries with larger populations. I'll therefore filter based on population size.

In [83]:
%%sql

SELECT *
  FROM facts
 WHERE population > 100000
 ORDER BY population_growth
 LIMIT 10;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
92,kv,Kosovo,10887,10887,0,1870981,None,None,None,None
67,gr,Greece,131957,130647,1310,10775643,0.01,8.66,11.09,2.32
72,gy,Guyana,214969,196849,18120,735222,0.02,15.59,7.32,8.06
157,lo,Slovakia,49035,48105,930,5445027,0.02,9.91,9.74,0.04
176,tn,Tonga,747,717,30,106501,0.03,23.0,4.85,17.84
143,rs,Russia,17098242,16377742,720500,142423773,0.04,11.6,13.69,1.69
64,gg,Georgia,69700,69700,0,4931226,0.08,12.74,10.82,2.7
108,mv,Maldives,298,298,0,393253,0.08,15.75,3.89,12.68
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
140,po,Portugal,92090,91470,620,10825309,0.09,9.27,11.02,2.67


### Observations

Many of the top countries that have a high population growth are located in Africa. These were also some of the same countries that also had a high birth rate as in the previous analysis. 

Many of the top countries with a low population growth are located in Europe. The only exception being the Maldives and Tonga, small island countries that both have high migration rates in common, and also Guyana located in South America. The largest country by far with a very low population growth is Russia.

# Summary statistics - water to land ratio

Looking at countries with the highest water to land ratio. 

In [84]:
%%sql

SELECT name, area_land, area_water, (area_land + area_water) AS total_area, 
area_land / ROUND(area_land + area_water, 2) * 100.0 AS land_pct,
area_water / ROUND(area_land + area_water, 2) * 100.0 AS water_pct
  FROM facts
 ORDER BY water_pct DESC
 LIMIT 10;

Done.


name,area_land,area_water,total_area,land_pct,water_pct
British Indian Ocean Territory,60,54340,54400,0.11029411764705882,99.88970588235294
Virgin Islands,346,1564,1910,18.1151832460733,81.8848167539267
Puerto Rico,8870,4921,13791,64.3173083895294,35.6826916104706
"Bahamas, The",10010,3870,13880,72.11815561959655,27.881844380403457
Guinea-Bissau,28120,8005,36125,77.840830449827,22.15916955017301
Malawi,94080,24404,118484,79.40312616049424,20.596873839505754
Netherlands,33893,7650,41543,81.58534530486484,18.41465469513516
Uganda,197100,43938,241038,81.77133895900232,18.228661040997686
Eritrea,101000,16600,117600,85.8843537414966,14.1156462585034
Liberia,96320,15049,111369,86.48726306243209,13.512736937567905


### Observations

The two British overseas colonies British Indian Ocean Territory and Virgin Islands by far have the highest water to land ratio. Puero Rico is third, with over 35% of water to land. This is then followed by the Bahamas. What all of the top countries have in common is that they are all islands. 

Another two countries high on the list are Guinea-Bissau and Malawi. Malawi's water to land ratio is accounted for by the large Lake Malawi that its land sits beside, and Guinea-Bissau is a coastal country that has many islands that jut out and therefore the waters surrounding these islands are part of its territory.  

The only European country (with the exception of overseas territories) to appear on the top 10 list is the Netherlands, a country consisting of several islands and coastal landmasses.

# Using python to obtain correlation values

To help direct analysis and find which columns are strongly correlated. 

In [85]:
#import python, create dataframe from data source
import pandas as pd

facts_py = pd.read_sql_table('facts', 'sqlite:///factbook.db') 

In [86]:
facts_corr = facts_py.corr()

In [87]:
facts_corr_high = facts_corr > 0.10

In [88]:
facts_corr[facts_corr_high]

,id,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
id,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.242882
area,NaN,1.000000,0.999607,0.855153,0.477199,NaN,NaN,NaN,NaN
area_land,NaN,0.999607,1.000000,0.839850,0.460009,NaN,NaN,NaN,NaN
area_water,NaN,0.855153,0.839850,1.000000,0.408571,NaN,NaN,NaN,NaN
population,NaN,0.477199,0.460009,0.408571,1.000000,NaN,NaN,NaN,NaN
population_growth,NaN,NaN,NaN,NaN,NaN,1.000000,0.784596,NaN,NaN
birth_rate,NaN,NaN,NaN,NaN,NaN,0.784596,1.000000,0.125131,NaN
death_rate,NaN,NaN,NaN,NaN,NaN,NaN,0.125131,1.000000,NaN
migration_rate,0.242882,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
